# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import pandas as pd
import hvplot.pandas
import requests
from requests.structures import CaseInsensitiveDict
import csv

# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,24.43,44,44,6.14,AU,1670133626
1,1,bredasdorp,-34.5322,20.0403,17.12,53,6,1.09,ZA,1670134070
2,2,punta arenas,-53.1500,-70.9167,5.06,75,40,5.66,CL,1670134070
3,3,pangai,-19.8000,-174.3500,27.32,79,100,5.08,TO,1670134071
4,4,mataura,-46.1927,168.8643,10.27,92,100,3.03,NZ,1670134071


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [79]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [61]:
# Narrow down cities that fit criteria and drop any results with null values
city_list_df = city_data_df.loc[(city_data_df['Max Temp'] < 27) & 
                                (city_data_df['Max Temp'] > 21) & 
                                (city_data_df['Wind Speed'] < 4.5) & 
                                (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
city_list_df = city_list_df.dropna()

# Display sample data
city_list_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,santiago,-33.4569,-70.6483,21.59,51,0,2.06,CL,1670133926
53,53,revelganj,25.7833,84.6667,22.95,33,0,2.79,IN,1670134084
162,162,nizwa,22.9333,57.5333,24.13,37,0,1.87,OM,1670133882
176,176,tezu,27.9167,96.1667,26.58,25,0,1.53,IN,1670134120
207,207,khandbari,27.3747,87.2039,21.57,48,0,2.27,NP,1670134130
272,272,pochutla,15.7432,-96.4661,22.68,57,0,1.75,MX,1670134151
276,276,la romana,18.4273,-68.9728,21.07,91,0,3.83,DO,1670134152
314,314,ormara,25.2088,64.6357,24.46,40,0,1.51,PK,1670134166
333,333,lahij,13.1667,44.5833,24.12,65,0,3.28,YE,1670134172
356,356,shahabad,27.6500,79.9500,22.77,28,0,2.35,IN,1670134179


### Step 3: Create a new DataFrame called `hotel_df`.

In [88]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = pd.DataFrame()
hotel_df = city_list_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Append the City information into city_data list
# ??

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
23,santiago,CL,-33.4569,-70.6483,51,
53,revelganj,IN,25.7833,84.6667,33,
162,nizwa,OM,22.9333,57.5333,37,
176,tezu,IN,27.9167,96.1667,25,
207,khandbari,NP,27.3747,87.2039,48,
272,pochutla,MX,15.7432,-96.4661,57,
276,la romana,DO,18.4273,-68.9728,91,
314,ormara,PK,25.2088,64.6357,40,
333,lahij,YE,13.1667,44.5833,65,
356,shahabad,IN,27.6500,79.9500,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [90]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
            "limit": "1",
            "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

# https://api.geoapify.com/v2/places?categories=public_transport.subway&bias=proximity:-0.07071648508463113,51.50848194136378&limit=20&apiKey=YOUR_API_KEY

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    # name_address = pd.json_normalize(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
santiago - nearest hotel: apart arturo prat
revelganj - nearest hotel: No hotel found
nizwa - nearest hotel: Nizwa Heritage Inn
tezu - nearest hotel: No hotel found
khandbari - nearest hotel: Guesthouse
pochutla - nearest hotel: Hotel El Patio
la romana - nearest hotel: Hotel Frano
ormara - nearest hotel: No hotel found
lahij - nearest hotel: No hotel found
shahabad - nearest hotel: No hotel found
flinders - nearest hotel: No hotel found
bhinga - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
23,santiago,CL,-33.4569,-70.6483,51,apart arturo prat
53,revelganj,IN,25.7833,84.6667,33,No hotel found
162,nizwa,OM,22.9333,57.5333,37,Nizwa Heritage Inn
176,tezu,IN,27.9167,96.1667,25,No hotel found
207,khandbari,NP,27.3747,87.2039,48,Guesthouse
272,pochutla,MX,15.7432,-96.4661,57,Hotel El Patio
276,la romana,DO,18.4273,-68.9728,91,Hotel Frano
314,ormara,PK,25.2088,64.6357,40,No hotel found
333,lahij,YE,13.1667,44.5833,65,No hotel found
356,shahabad,IN,27.6500,79.9500,28,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [92]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.65,
    color = "City",
    hover_cols = "all"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)